In [1]:
import tensorflow as tf

# Eager execution

### Example illustration

In [2]:
a=tf.constant(2)
b=tf.constant(3)
add=a+b
mul=a*b
print("a + b =",add)
print("a * b =",mul)

#In this example, a + b and a * b are executed immediately, and their results are available as soon as they are computed.

a + b = tf.Tensor(5, shape=(), dtype=int32)
a * b = tf.Tensor(6, shape=(), dtype=int32)


# Function/Graph - tf.function()

**Performance Optimization:** Graphs are optimized for performance, allowing for faster execution compared to eager mode.
**Serialization:** Graphs can be serialized, which is useful for saving and deploying models.
**Device Placement:** Graphs can be executed on various devices (CPU, GPU, TPU) without modification.
**Parallel Execution:** Operations in the graph can be parallelized and optimized by the TensorFlow runtime.

In [4]:
#example illustartion
@tf.function
def add(a,b):
    return a+b

x,y=20,30
res=add(x,y)
print("resukt = ",res)

resukt =  tf.Tensor(50, shape=(), dtype=int32)


# GradientTape() - Automatic diffrentiation

**tf.GradientTape:** This context manager records operations for automatic differentiation.

**tape.watch(tensor):** Manually watches a tensor. This is necessary for tensors that are not tf.Variable objects, such as constants.

**tape.gradient(target, sources):** Computes the gradient of target (usually a loss function) with respect to sources (usually model parameters).

**tape.stop_recording():** Temporarily stops recording operations.

**tape.reset():** Resets the tape, discarding all recorded operations.

**tape.enter()** and **tape.exit(exc_type, exc_value, traceback):** These methods are used internally to start and stop the gradient recording when entering and exiting the context manager.

**tape.batch_jacobian(target, source):** Computes the Jacobian matrix of target with respect to source.

### Example-1 - with constant

Before diff : f(x)=x^2+3x+2

After diff with respect to x : f'(x)=2x+3

At x=2 : 

f(2.0)=16

f'(2.0)=15

In [8]:
# Define a function for which we want to compute the gradient
@tf.function
def f(x):
    return x**3+3*x+2

# Define the variable
# Note: that this is a tf.Tensor object, not a tf.Variable.
x=tf.constant(2.0)

"""
with tf.GradientTape() as tape: This context manager starts recording operations.
tape.watch(x): Explicitly tells the tape to watch the tensor x for gradient computation.
y = f(x): Computes the function value 
"""
@tf.function
def compute_gradient(x):
    
    #This context manager starts recording operations
    with tf.GradientTape() as tape: 
        tape.watch(x) # Manually watch x because it's not a tf.Variable
        y=f(x) # Define the function
        
    # Compute the gradient of y with respect to x
    return tape.gradient(y,x)

f_x=f(x)
dy_dx=compute_gradient(x)

print("f(x)=",f_x)
print("f'(x)=",dy_dx)



f(x)= tf.Tensor(16.0, shape=(), dtype=float32)
f'(x)= tf.Tensor(15.0, shape=(), dtype=float32)


### Example 2-  with variables

y=w*x+b


dy_dw=x

dy_db=1


In [9]:
w = tf.Variable(2.0)
b = tf.Variable(3.0)
x = tf.constant(1.0)

In [11]:
@tf.function
def f(x):
    return w*x+b
@tf.function
def compute_gradient(x):
    #This context manager starts recording operations
    with tf.GradientTape() as tape:
        #define function
        y=f(x)
    return tape.gradient(y,[w,b])

gradients=compute_gradient(x)
print("Gradient of y with respect to w:", gradients[0].numpy())
print("Gradient of y with respect to b:", gradients[1].numpy())

Gradient of y with respect to w: 1.0
Gradient of y with respect to b: 1.0


In [12]:
#Note :  I above example we didnt use the tape.watch(x) 
#reason:
#dy_dw and dy_db we are computing these two are declared with tf.Variable() so dont have to use that watch

### Example 3 - Temporarily stop recording operations

In [16]:
x=tf.constant(2.0)
@tf.function
def f(x):
    return x**2+3*x+2

@tf.function
def compute_gradient(x):
    #This context manager starts recording operations
    with tf.GradientTape() as tape:
        tape.watch(x)
        with tape.stop_recording():
            y=f(x) # This won't be recorded
    # Compute gradient (will be None because recording was stopped)
    return tape.gradient(y,x)

dy_dx=compute_gradient(x)
print("Gradient when recording was stopped:", dy_dx)

Gradient when recording was stopped: None


### Example 4 - Compute Jacobian

In [27]:
x = tf.constant([[1.0, 2.0], [3.0, 4.0]])

@tf.function
def compute_jacobian(x):
     #This context manager starts recording operations
    with tf.GradientTape() as tape:
        tape.watch(x)
        #defining the function
        y=tf.reduce_sum(x**2,axis=1)
        y = tf.reshape(y, (-1, 1))
    #computing jacobian
    return tape.batch_jacobian(y,x)

jacob=compute_jacobian(x)
print("Jacobian matrix:\n", jacob.numpy())

Jacobian matrix:
 [[[2. 4.]]

 [[6. 8.]]]
